<a href="https://colab.research.google.com/github/catPotat/rn-hg-gpt2-playground/blob/main/Huggingface_GPT_2_Evaluating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Initialize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pip install transformers
!pip install nlp
# !pip install datasets
!pip install python-dotenv
try:
    from google.colab import output
    output.clear()
except: pass
import os
from dotenv import load_dotenv
!cp 'drive/MyDrive/.env.gdrive' '.env'
load_dotenv()

cp: cannot stat 'drive/MyDrive/.env.gdrive': No such file or directory


True

In [5]:
MODEL_NAME = 'gpt2'
MODEL_SAVE_PATH = ''
TOK_PATH = MODEL_SAVE_PATH + "tok/"

## PPL

Pretrained tokenizer

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 128
# tokenizer.set_truncation_and_padding(max_length=512)
tokenizer.encode(" 9 I J J : : J.J J J J J ")

[860, 314, 449, 449, 1058, 1058, 449, 13, 41, 449, 449, 449, 449, 220]

In [7]:
from nlp import load_dataset
from tqdm.notebook import tqdm
from transformers import GPT2LMHeadModel

test = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer('\n\n'.join(test['text']), return_tensors='pt')
stride = 512

Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 128). Running this sequence through the model will result in indexing errors


In [9]:
test.__dir__()

['_info',
 '_split',
 '_indexes',
 '_data',
 '_data_files',
 '_format_type',
 '_format_kwargs',
 '_format_columns',
 '_output_all_columns',
 '__module__',
 '__doc__',
 '__init__',
 'from_file',
 'from_buffer',
 'from_pandas',
 'from_dict',
 'data',
 'cache_files',
 'columns',
 'nbytes',
 'num_columns',
 'num_rows',
 'column_names',
 'shape',
 'drop',
 'unique',
 'dictionary_encode_column',
 'flatten',
 'cast_',
 'remove_column_',
 'rename_column_',
 '__len__',
 '__iter__',
 '__repr__',
 'format',
 'formated_as',
 'set_format',
 'reset_format',
 '_convert_outputs',
 '_unnest',
 '_nest',
 '_getitem',
 '__getitem__',
 'cleanup_cache_files',
 '_get_cache_file_path',
 'map',
 'filter',
 'select',
 'sort',
 'shuffle',
 'export',
 'train_test_split',
 'shard',
 'add_faiss_index',
 'add_faiss_index_from_external_arrays',
 'add_elasticsearch_index',
 'info',
 'split',
 'builder_name',
 'citation',
 'config_name',
 'dataset_size',
 'description',
 'download_checksums',
 'download_size',
 'featur

In [ ]:
to_compare = [
    MODEL_NAME,
    MODEL_SAVE_PATH,
]
for root, dirs, files in os.walk(MODEL_SAVE_PATH, topdown=False):
    for name in dirs:
        print(name)
        model_path = os.path.join(root, name)
        to_compare.append(model_path)

In [ ]:
for model_path in to_compare:
    model = GPT2LMHeadModel.from_pretrained(model_path)
    max_length = model.config.n_positions
    model.to("cuda")

    lls = []
    for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i    # may be different from stride on last loop
        input_ids = encodings.input_ids[:,begin_loc:end_loc].to("cuda")
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            log_likelihood = outputs[0] * trg_len

        lls.append(log_likelihood)

    ppl = torch.exp(torch.stack(lls).sum() / end_loc)
    print(ppl)

## Empirical

In [ ]:
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model=MODEL_SAVE_PATH + '',
                     tokenizer=tokenizer)
set_seed(6110)

In [ ]:
generator("", max_length=50, num_return_sequences=5)

https://github.com/huggingface/transformers/issues/1407

## iGPT

In [ ]:
CONTRAST = 64
BRIGHTNESS = 24
def reconstruct_from(text, path, img_w=0, img_h=0):
    lines = text.split(LINE_SPLIT_CHAR)
    width = img_w if img_w else len( max(lines, key=len) )
    height = img_h if img_h else len(lines)
    pixels = []
    for y in range(height):
        for x in range(width):
            pixels.append((0, 0, 0, 0))
    for index, line in enumerate(lines):
        line = line.split(PIXEL_SPLIT_CHAR) if PIXEL_SPLIT_CHAR  else line
        for x in range(len(line) - 1):
            s = line[x + 1]
            try:
                if s == '~': continue
                r = g = b = 0
                c = ord(s[0])-CHAR_CODE_SHIFT
                b = (c & 3) * CONTRAST + BRIGHTNESS
                c = c >> 2
                g = (c & 3) * CONTRAST + BRIGHTNESS
                c = c >> 2
                r = (c & 3) * CONTRAST + BRIGHTNESS
                i = (index*width) + x
                pixels[i] = (r, g, b, 255)
            except: 
                print('fuck')
    image = Image.new('RGBA', (width, height))
    image.putdata(pixels)
    image.save(path)
    return image
# reconstruct_from(text_entries[-1], "test.png")

In [ ]:
genner = generator("", max_length=950, num_return_sequences=5)

In [ ]:
i = 4

In [ ]:
reconstruct_from(genner[i]['generated_text'].strip(), "generated.png")

In [ ]:
reconstruct_from(genner[i]['generated_text'].strip(), "generated.png", IMG_WIDTH, IMG_HEIGHT)